## Alphabet Soup Charity with Optimization

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head(3)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts = classification_counts[classification_counts>1]
counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df)
app_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = app_dummies.drop(columns="IS_SUCCESSFUL")
y = app_dummies["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

Use 3 layers

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=80
hidden_nodes_layer2=40
hidden_nodes_layer3=20

nn = tf.keras.models.Sequential()


# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 40)                3240      
                                                                 
 dense_2 (Dense)             (None, 20)                820       
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 7,601
Trainable params: 7,601
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

In [14]:
# Train the model
fitModel = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 11s 6ms/step - loss: 0.5706 - accuracy: 0.7206
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5552 - accuracy: 0.7298
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5515 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5508 - accuracy: 0.7312
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7321
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5467 - accuracy: 0.7348
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5466 - accuracy: 0.7331
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accur

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5621 - accuracy: 0.7290 - 574ms/epoch - 2ms/step
Loss: 0.5621073246002197, Accuracy: 0.7289795875549316


## Opimization of the hyperparams using Keras-tuner



In [16]:
pip install -q keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.1 MB/s eta 0:00:00


In [17]:
# import the keras-tuner modules
import keras_tuner as kt

In [18]:
# make the function for hypertuning the deep learning models

def makeDLModel(hp):

  # set up the sequential model
  nnModel = tf.keras.models.Sequential()

  # set up the choices for the activation functions needed in the hidden layers
  activationList = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

  # define the number of input features
  input_features = len( X_train_scaled[0])

  # loop through a number of attributes on the first layer, loop through 1 - 20 neurons on next layer

  nnModel.add(
      tf.keras.layers.Dense(
          input_dim=input_features,
          activation=activationList,
          units=hp.Int('first_units', min_value=1, max_value=20)
      )
  )

  # test different numbers of layers as well as neurons on those hidden layers
  for num in range(hp.Int('num_layers', 1, 21)):

    # add the layer (or layers) with a neuron (or neurons) on each layer
    nnModel.add(
        tf.keras.layers.Dense(
            activation=activationList,
            units=hp.Int(
                'units_' + str(num),
                min_value=1,
                max_value=20
            )
        )
    )

  # set up the output layer (1 node)
  nnModel.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  # compile the model
  nnModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  # return the model to used in the iteration
  return nnModel

In [19]:
# use kt Hyperband function to run the optimizer
tunerModel = kt.Hyperband(
    makeDLModel,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2
)

In [20]:
# use the .search() function to find the best combo of hyperparameters
tunerModel.search(X_train_scaled, y_train, epochs=30, validation_data=[X_test_scaled, y_test])

Trial 180 Complete [00h 01m 56s]
val_accuracy: 0.5292128324508667

Best val_accuracy So Far: 0.7337609529495239
Total elapsed time: 01h 32m 20s


In [21]:
# get the best hyperparameters
tunerModel.get_best_hyperparameters(1)[0].values

{'activation': 'sigmoid',
 'first_units': 7,
 'num_layers': 7,
 'units_0': 8,
 'units_1': 18,
 'units_2': 6,
 'units_3': 9,
 'units_4': 19,
 'units_5': 10,
 'units_6': 15,
 'units_7': 7,
 'units_8': 18,
 'units_9': 8,
 'units_10': 10,
 'units_11': 2,
 'units_12': 14,
 'units_13': 2,
 'units_14': 16,
 'units_15': 18,
 'units_16': 14,
 'units_17': 7,
 'units_18': 14,
 'units_19': 10,
 'units_20': 19,
 'tuner/epochs': 30,
 'tuner/initial_epoch': 10,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0139'}

In [22]:
# use get_best_models to evaluate the model loss and accuracy
bestDLModel = tunerModel.get_best_models(1)[0]

DLModel02Loss, DLModel02Accuracy = bestDLModel.evaluate(X_test_scaled, y_test)

268/268 [==============================] - 1s 2ms/step - loss: 0.5747 - accuracy: 0.7338


In [23]:
# import the load and save model modules from keras.models
from tensorflow.keras.models import load_model, save_model

In [24]:
# export the best deep learning model as an H5 file
bestDLModel.save('AlphabetSoupCharity_Optimization.h5')